In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.sql import functions as F

In [2]:
conf = SparkConf().setMaster("local[4]").setAppName("transport")
sc = SparkContext(conf=conf)
sqlContext = SQLContext(sc)

In [3]:
path = "file:///home/nicolas/github/improve_transport/datasets/data.parquet/"

In [4]:
df = sqlContext.read.parquet(path)

In [5]:
df.show()

+-------------------+-------------+------------------+-----------+-----------+--------------------+
|       fechahoratrx|codigoentidad|     nombreentidad|codigositio|nombresitio|          nrotarjeta|
+-------------------+-------------+------------------+-----------+-----------+--------------------+
|2019-02-28 21:00:15|           13|       U6 - Redbus|       2596|    CJRC-35|612d6a01a6158dd63...|
|2019-02-28 21:02:39|            5|U5 - Metropolitana|       7660|    FLXG-38|eb7a60b4eb2e7b0a9...|
|2019-02-28 21:11:25|            5|U5 - Metropolitana|      11338|    FLXT-94|b4a501a26eaab4f8a...|
|2019-02-28 21:14:22|           13|       U6 - Redbus|       2596|    CJRC-35|a727f39a7a85cb6c1...|
|2019-02-28 21:17:02|            5|U5 - Metropolitana|      14379|    BJFJ-15|dcb9033eb5920016b...|
|2019-02-28 21:17:07|            5|U5 - Metropolitana|      14379|    BJFJ-15|8647ac944c4fa5928...|
|2019-02-28 21:20:12|           13|       U6 - Redbus|       2596|    CJRC-35|4339814f270bfab3d...|


In [6]:
df.printSchema()

root
 |-- fechahoratrx: timestamp (nullable = true)
 |-- codigoentidad: long (nullable = true)
 |-- nombreentidad: string (nullable = true)
 |-- codigositio: long (nullable = true)
 |-- nombresitio: string (nullable = true)
 |-- nrotarjeta: string (nullable = true)



In [7]:
df.count()

2433466

In [8]:
df.groupBy("nrotarjeta").count().orderBy("count", ascending=False).show()

+--------------------+-----+
|          nrotarjeta|count|
+--------------------+-----+
|f4221acc3ec26146b...|   25|
|0b286ef4a356304ef...|   24|
|f1cc16d8093662c10...|   21|
|e21255cc625840b88...|   20|
|4650034e1690026c9...|   20|
|e9904008854abc186...|   20|
|9bc9ad4835dae1ca9...|   20|
|ba602fc72c91fc9bd...|   20|
|5b040cedf7146d46b...|   18|
|ca3454d9131f8b02a...|   18|
|05ce66ed7948936ac...|   18|
|b5a4026d77cad264d...|   18|
|c58c3d4bd2924126d...|   18|
|f975001ffe1b87792...|   18|
|198ff7943b4d2a61b...|   17|
|12b382c2f5297029b...|   17|
|64db36ddb326c94b0...|   17|
|8052f93490092aa4a...|   16|
|8697342494cdf1400...|   16|
|55070b9c28e40eed0...|   16|
+--------------------+-----+
only showing top 20 rows



In [9]:
df.createOrReplaceTempView('df')

In [10]:
query = """
    SELECT nrotarjeta, nombreentidad, COUNT(*) as count
    FROM df
    GROUP BY nrotarjeta, nombreentidad
"""
result = sqlContext.sql(query)
result.show()

+--------------------+------------------+-----+
|          nrotarjeta|     nombreentidad|count|
+--------------------+------------------+-----+
|6cbcf07f6b9502933...|       U6 - Redbus|    1|
|494c93b4f91813b1a...|U5 - Metropolitana|    1|
|d4d5aefb798c8a9fe...|U5 - Metropolitana|    1|
|b0bb29fde7565da7c...|U5 - Metropolitana|    1|
|4cd7438eace49cfd7...|       U6 - Redbus|    2|
|a3b629b2a227e9e23...|U5 - Metropolitana|    1|
|33447cc0954101c9a...|       U6 - Redbus|    1|
|84f192fe4e3298eb1...|U5 - Metropolitana|    1|
|8f5a57848989c0f00...|U5 - Metropolitana|    1|
|27689d10932716361...|U5 - Metropolitana|    1|
|dbad6f8dd5196036b...|         U3 - Vule|    1|
|7709d1be22bf32852...|U5 - Metropolitana|    1|
|668a196b4437e23fc...|U5 - Metropolitana|    2|
|d9ecd35d5c6d7617d...|       U6 - Redbus|    1|
|cef59dd4cfa92d770...|U5 - Metropolitana|    2|
|b56b54f26203e2162...|U5 - Metropolitana|    1|
|2c3e783d7f8aba7d7...|         U3 - Vule|    1|
|b7017908a27ee4e25...|U5 - Metropolitana

In [11]:
query = """
    SELECT nrotarjeta, nombreentidad, count
    FROM (
        SELECT nrotarjeta, nombreentidad, count,
            DENSE_RANK() OVER(
                PARTITION BY nombreentidad
                ORDER BY count DESC
            ) AS rank
        FROM (
            SELECT nrotarjeta, nombreentidad, COUNT(*) as count
            FROM df
            GROUP BY nrotarjeta, nombreentidad
            ORDER BY count DESC
        )
    )
    WHERE rank <= 2
    ORDER BY count DESC
"""
result = sqlContext.sql(query)
result.show()

+--------------------+------------------+-----+
|          nrotarjeta|     nombreentidad|count|
+--------------------+------------------+-----+
|0b286ef4a356304ef...|        METRO - OT|   24|
|f1cc16d8093662c10...|        METRO - OT|   21|
|c909309d0c5c313ea...|U5 - Metropolitana|   11|
|3ed0d4f78603b46e2...|      U4 - Express|   11|
|ae472f38ce14d7347...|U5 - Metropolitana|   11|
|4e9f3beb896b8273b...|      Tren Central|   10|
|60c31e815e3f380d8...|U5 - Metropolitana|   10|
|314f4188e4c591a28...|       U2 - Su Bus|   10|
|5b040cedf7146d46b...|       U6 - Redbus|   10|
|f947f0bcc72db9ad8...|U5 - Metropolitana|   10|
|45d4c63334c9dacd2...|       U2 - Su Bus|    9|
|27d9e5f3e90685047...|       U2 - Su Bus|    9|
|0c298d52142b0101d...|      Tren Central|    9|
|37ce06df1ade640f4...|         U3 - Vule|    9|
|a48e9d07b409708c5...|       U2 - Su Bus|    9|
|862447def346871c4...|       U2 - Su Bus|    9|
|f45d87cd753b25fae...|       U2 - Su Bus|    9|
|37c17fc6843cd5a7d...|          U7 - STP

In [12]:
result = result.collect()

In [13]:
result

[Row(nrotarjeta='0b286ef4a356304ef4d0149b49f77cdb2c9ba795b7e516a4309f68c7bb9d3e81', nombreentidad='METRO - OT', count=24),
 Row(nrotarjeta='f1cc16d8093662c10a97f3e23812ab664743990a983f6d30662bc190a3973a36', nombreentidad='METRO - OT', count=21),
 Row(nrotarjeta='3ed0d4f78603b46e29e8640e18bb1020734f7721ffb8113c3ece0c98659c47e6', nombreentidad='U4 - Express', count=11),
 Row(nrotarjeta='c909309d0c5c313ea66a1d98ac8e8f579130935dfa2bf1b44dcd2746721ff0c8', nombreentidad='U5 - Metropolitana', count=11),
 Row(nrotarjeta='ae472f38ce14d7347ef36161197e7801b799d6d8257e22c43653ed86fa53bead', nombreentidad='U5 - Metropolitana', count=11),
 Row(nrotarjeta='314f4188e4c591a28cc19ae26c3d092a6cbaf5a19c28aabf7938596b1be75aeb', nombreentidad='U2 - Su Bus', count=10),
 Row(nrotarjeta='4e9f3beb896b8273bbac5bf11437b3f59ca2fcb08b8eea267b79e360035198e0', nombreentidad='Tren Central', count=10),
 Row(nrotarjeta='5b040cedf7146d46bb3cea4c5c1971be71f6379f15e3e29fe534c5e17a4d2970', nombreentidad='U6 - Redbus', count